In [1]:
import os
import numpy as np
import pandas as pd
import scipy.io as sio
import matplotlib.pyplot as plt

from scipy.signal import stft
from collections import defaultdict
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras import layers, models

In [ ]:
# Paths
MAT_DIR = r'D:\ECG_model\Data'    # contains A0001.mat ...
CSV_PATH = r'D:\ECG_model\REFERENCE.csv'       # Recording, First_label
IMG_DIR = "ecg_images_2000"

# ECG settings
FS = 500
FIXED_LEN = 7500
WINDOW = 2000        # 4 seconds
STRIDE = 1000        # 50% overlap
LEADS = [0, 1, 2]    # Lead-1, Lead-2, Lead-3

# Image / model
IMG_SIZE = (224, 224)
NUM_CLASSES = 9
BATCH_SIZE = 32
EPOCHS = 30


In [3]:
labels_df = pd.read_csv(CSV_PATH)

label_map = {
    row["Recording"]: row["First_label"] - 1
    for _, row in labels_df.iterrows()
}


In [4]:
def fix_length(sig, length=7500):
    if len(sig) >= length:
        return sig[:length]
    return np.pad(sig, (0, length - len(sig)))


In [5]:
def tf_image(signal):
    f, t, Zxx = stft(signal, fs=FS, nperseg=256)
    img = np.abs(Zxx)
    img = (img - img.min()) / (img.max() - img.min() + 1e-8)
    return img


In [6]:
os.makedirs(IMG_DIR, exist_ok=True)
for c in range(NUM_CLASSES):
    os.makedirs(os.path.join(IMG_DIR, f"class_{c+1}"), exist_ok=True)

for file in os.listdir(MAT_DIR):

    rec_id = file.replace(".mat", "")
    if rec_id not in label_map:
        continue

    label = label_map[rec_id]
    mat = sio.loadmat(os.path.join(MAT_DIR, file))
    ecg = mat["ECG"][0][0]["data"]

    leads = [fix_length(ecg[l]) for l in LEADS]

    win_id = 0
    for start in range(0, FIXED_LEN - WINDOW + 1, STRIDE):

        channels = []
        for sig in leads:
            w = sig[start:start + WINDOW]
            channels.append(tf_image(w))

        rgb = np.stack(channels, axis=-1)
        rgb = tf.image.resize(rgb, IMG_SIZE).numpy()

        save_path = os.path.join(
            IMG_DIR,
            f"class_{label+1}",
            f"{rec_id}_win{win_id}.png"
        )

        plt.imsave(save_path, rgb)
        win_id += 1


In [7]:
train_dir, val_dir, test_dir = "train", "val", "test"

for d in [train_dir, val_dir, test_dir]:
    for c in range(NUM_CLASSES):
        os.makedirs(os.path.join(d, f"class_{c+1}"), exist_ok=True)

groups = defaultdict(list)

for c in range(NUM_CLASSES):
    for f in os.listdir(os.path.join(IMG_DIR, f"class_{c+1}")):
        ecg_id = f.split("_win")[0]
        groups[(ecg_id, c)].append(f)

ecgs = list(groups.keys())
train_ecg, temp = train_test_split(ecgs, test_size=0.3, random_state=42)
val_ecg, test_ecg = train_test_split(temp, test_size=0.5, random_state=42)


In [8]:
def copy_ecgs(ecg_list, target):
    for ecg_id, cls in ecg_list:
        for img in groups[(ecg_id, cls)]:
            src = os.path.join(IMG_DIR, f"class_{cls+1}", img)
            dst = os.path.join(target, f"class_{cls+1}", img)
            tf.io.gfile.copy(src, dst, overwrite=True)

copy_ecgs(train_ecg, train_dir)
copy_ecgs(val_ecg, val_dir)
copy_ecgs(test_ecg, test_dir)


In [9]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir, image_size=IMG_SIZE, batch_size=BATCH_SIZE
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    val_dir, image_size=IMG_SIZE, batch_size=BATCH_SIZE
)

test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    test_dir, image_size=IMG_SIZE, batch_size=BATCH_SIZE, shuffle=False
)


Found 14886 files belonging to 9 classes.
Found 3192 files belonging to 9 classes.
Found 3192 files belonging to 9 classes.


In [10]:
base_model = EfficientNetB0(
    include_top=False,
    weights="imagenet",
    input_shape=(224,224,3)
)

base_model.trainable = True
for layer in base_model.layers[:200]:
    layer.trainable = False

inputs = layers.Input(shape=(224,224,3))
x = tf.keras.applications.efficientnet.preprocess_input(inputs)
x = base_model(x)
x = layers.GlobalAveragePooling2D()(x)
x = layers.BatchNormalization()(x)
outputs = layers.Dense(NUM_CLASSES, activation="softmax")(x)

model = models.Model(inputs, outputs)


In [11]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-5),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)


In [12]:
model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor="val_loss",
            patience=8,
            restore_best_weights=True
        )
    ]
)


Epoch 1/30
466/466 [==============================] - 60s 110ms/step - loss: 2.4239 - accuracy: 0.2022 - val_loss: 2.1913 - val_accuracy: 0.2638
Epoch 2/30
466/466 [==============================] - 52s 111ms/step - loss: 2.0687 - accuracy: 0.3102 - val_loss: 2.0812 - val_accuracy: 0.3195
Epoch 3/30
466/466 [==============================] - 52s 111ms/step - loss: 1.9178 - accuracy: 0.3521 - val_loss: 1.9975 - val_accuracy: 0.3434
Epoch 4/30
466/466 [==============================] - 52s 112ms/step - loss: 1.8172 - accuracy: 0.3862 - val_loss: 1.9471 - val_accuracy: 0.3615
Epoch 5/30
466/466 [==============================] - 52s 112ms/step - loss: 1.7438 - accuracy: 0.4080 - val_loss: 1.9059 - val_accuracy: 0.3781
Epoch 6/30
466/466 [==============================] - 52s 112ms/step - loss: 1.6805 - accuracy: 0.4312 - val_loss: 1.8649 - val_accuracy: 0.3925
Epoch 7/30
466/466 [==============================] - 53s 113ms/step - loss: 1.6264 - accuracy: 0.4425 - val_loss: 1.8474 - val_ac

In [13]:
probs, y_true = [], []
paths = test_ds.file_paths

for imgs, labels in test_ds:
    p = model.predict(imgs, verbose=0)
    probs.extend(p)
    y_true.extend(labels.numpy())

ecg_probs = defaultdict(list)
ecg_true = {}

for path, p, t in zip(paths, probs, y_true):
    ecg_id = os.path.basename(path).split("_win")[0]
    ecg_probs[ecg_id].append(p)
    ecg_true[ecg_id] = t

final_pred, final_true = [], []

for ecg_id, plist in ecg_probs.items():
    avg_p = np.mean(plist, axis=0)
    final_pred.append(np.argmax(avg_p))
    final_true.append(ecg_true[ecg_id])

print("ECG-Level Accuracy:",
      accuracy_score(final_true, final_pred))

print("\nECG-Level Classification Report:\n")
print(classification_report(
    final_true,
    final_pred,
    labels=list(range(9)),
    digits=2
))


ECG-Level Accuracy: 0.49624060150375937

ECG-Level Classification Report:

              precision    recall  f1-score   support

           0       0.58      0.54      0.56        91
           1       0.50      0.69      0.58        80
           2       0.42      0.56      0.48        48
           3       0.86      0.43      0.57        14
           4       0.54      0.65      0.59       122
           5       0.15      0.06      0.08        52
           6       0.57      0.31      0.41        51
           7       0.44      0.47      0.45        60
           8       0.11      0.07      0.09        14

    accuracy                           0.50       532
   macro avg       0.46      0.42      0.42       532
weighted avg       0.48      0.50      0.48       532

